In [1]:
import os

In [6]:
%pwd

'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd


'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [3]:
from cnnClassifier.constants import *
import os
from cnnClassifier.utils.common import read_yaml, create_directories,save_json
from cnnClassifier.entity.config_entity import (DataIngestionConfig,
                                                PrepareBaseModelConfig,
                                                TrainingConfig,
                                                EvaluationConfig)


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        if "prepare_model" not in self.config:
            raise KeyError("Key 'prepare_model' is missing from config.yaml!")
        config = self.config.prepare_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
        root_dir=Path(config.root_dir),
        base_model_path=Path(config.base_model_path),
        updated_base_model_path=Path(config.updated_base_model_path),
        params_image_size=[256, 256, 3],  
        params_learning_rate=self.params.LEARNING_RATE,
        params_include_top=False,  
        params_weights=None,  
        params_classes=self.params.CLASSES
    )

        return prepare_base_model_config

c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation
from pathlib import Path
from cnnClassifier.entity.config_entity import PrepareBaseModelConfig
from cnnClassifier.logger import logger  

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        """Defines and saves the custom CNN model."""
        model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(256, 256, 3)),
        BatchNormalization(),
        MaxPooling2D(),

        # Block 2
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(),

        # Block 3
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.3),

        # Block 4
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.4),

        # Classification Head
        Flatten(),
        Dense(256, activation='relu', kernel_initializer="he_normal"),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])

        # Save the model using the newly added method
        self.save_model(path=self.config.base_model_path, model=model)

    def save_model(self, path: Path, model: tf.keras.Model):
        """Save the trained model to a given path."""
        model.save(path)
        model.summary()
        logger.info(f"Model saved successfully at: {path}")

            
            


In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
        

    # Create and save the custom CNN model
    prepare_base_model.get_base_model()
    logger.info("Custom CNN model created successfully.")
except Exception as e:
    raise e

[2025-03-27 10:44:07,738: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-27 10:44:07,747: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 10:44:07,749: INFO: common: created directory at: artifacts]
[2025-03-27 10:44:07,750: INFO: common: created directory at: artifacts/prepare_model]
[2025-03-27 10:44:08,028: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 32)     128       
 ormalization)                                                   
                                                